In [1]:
from __future__ import division
from __future__ import print_function
from __future__ import absolute_import

In [2]:
import os
import pandas as pd
import tensorflow as tf
import google.datalab.bigquery as dlbq
tf.__version__

'1.12.0'

---
##### From Jupyter notebook to python package
From exploration to production.

This little tool dumps a given function to a file with the same name in a certain package

In [3]:
PACKAGE="./train"
from tools import make_src_dumper
write_py = make_src_dumper(PACKAGE)

---
# Hypotheses

#### The Catalogue of hypotheses. 
Import this function and choose by key in your model_fn

In [4]:
def make_hypotheses():

    from train.hypothesis_wd1 import hypothesis_wd1
    from train.hypothesis_linear import hypothesis_linear
    
    return {
        'wide_and_deep': hypothesis_wd1,
        'linear': hypothesis_linear 
    }
    
write_py(make_hypotheses)

'make_hypotheses written to ./train/make_hypotheses.py.'

#### Linear model

In [5]:
def hypothesis_linear(features, feature_columns, options):
    
    import tensorflow as tf
    from train.train_tools import weight_summary

    with tf.name_scope('Linear'):
    
        all_feature_columns = feature_columns['wide'] + feature_columns['deep']

        input_layer = tf.feature_column.input_layer( 
            features, feature_columns=all_feature_columns)

        out = tf.layers.dense(input_layer, 1, activation=None)
        weight_summary(out)
    
    return out
    
write_py(hypothesis_linear)

'hypothesis_linear written to ./train/hypothesis_linear.py.'

#### Deep and wide

In [6]:
def hypothesis_wd1(features, feature_columns, options):
    
    import tensorflow as tf

    
    wide_input_layer = tf.feature_column.input_layer( 
        features, feature_columns=feature_columns['wide'])

    deep_input_layer = tf.feature_column.input_layer( 
        features, feature_columns=feature_columns['deep'])

    # 40 x 20
    h1 = tf.layers.dense(deep_input_layer, 20, activation='relu')
    
    # 20 x 10
    h2 = tf.layers.dense(h1, 10, activation='relu')

    # 10 x 1
    o1 = tf.layers.dense(h2, 1, activation=None)

    o2 = tf.layers.dense(wide_input_layer, 1, activation=None)
    
    o = tf.concat([o1, o2], axis=1)
    
    hypothesis = tf.layers.dense(o, 1, activation=None)

    return hypothesis
    
write_py(hypothesis_wd1)

'hypothesis_wd1 written to ./train/hypothesis_wd1.py.'